# GO Subset Analysis

Analyze GO ribbons, exploring the degree of overlap between ribbon terms

In [1]:
# Basic OAK setup
from oaklib.datamodels.vocabulary import IS_A, PART_OF
from oaklib import get_adapter

go = get_adapter("sqlite:obo:go")

In [2]:
import pandas as pd
import numpy as np

In [7]:
# TODO: add more variables here
HUMAN = "NCBITaxon:9606"
MOUSE = "NCBITaxon:10090"
YEAST = "NCBITaxon:559292"
FLY = "NCBITaxon:7227"
ribbons = {"goslim_agr": [HUMAN, MOUSE, YEAST],
           #"goslim_euk_cellular_processes_ribbon": [HUMAN, MOUSE, YEAST],
           "goslim_flybase_ribbon": [FLY],
           }

In [8]:
from oaklib.interfaces.association_provider_interface import AssociationProviderInterface

_amigos = {}
def get_amigo(taxon) -> AssociationProviderInterface:
    if taxon not in _amigos:
        _amigos[taxon] = get_adapter(f"amigo:{taxon}")
    return _amigos[taxon]


In [9]:
_ribbon_terms = {}
def get_ribbon_terms(ribbon):
    if ribbon not in _ribbon_terms:
        _ribbon_terms[ribbon] = list(go.subset_members(ribbon))
    return _ribbon_terms[ribbon]

In [10]:

def fetch_matrix(taxon, ribbon):
    amigo = get_amigo(taxon)
    terms = get_ribbon_terms(ribbon)
    coassocs = list(amigo.association_pairwise_coassociations(terms, terms, include_entities=False))
    return coassocs

In [11]:
coassocs = fetch_matrix(FLY, "goslim_flybase_ribbon")

[PairwiseCoAssociation(object1='GO:0000003', object2='GO:0000003', object1_label=None, object2_label=None, number_subjects_in_common=1733, proportion_subjects_in_common=1.0, number_subjects_in_union=1733, number_subject_unique_to_entity1=0, number_subject_unique_to_entity2=0, subjects_in_common=[], associations_for_subjects_in_common=[]),
 PairwiseCoAssociation(object1='GO:0002376', object2='GO:0000003', object1_label=None, object2_label=None, number_subjects_in_common=78, proportion_subjects_in_common=0.03989769820971867, number_subjects_in_union=1955, number_subject_unique_to_entity1=222, number_subject_unique_to_entity2=1655, subjects_in_common=[], associations_for_subjects_in_common=[]),
 PairwiseCoAssociation(object1='GO:0002376', object2='GO:0002376', object1_label=None, object2_label=None, number_subjects_in_common=300, proportion_subjects_in_common=1.0, number_subjects_in_union=300, number_subject_unique_to_entity1=0, number_subject_unique_to_entity2=0, subjects_in_common=[], a

In [28]:
roots = {
    "biological_process": "GO:0008150",
    "cellular_component": "GO:0005575",
    "molecular_function": "GO:0003674",
}
_terms_to_roots = {}
def get_term_category(term_id: str):
    if term_id not in _terms_to_roots:
        ancs = list(go.ancestors(term_id, predicates=[IS_A]))
        for root_name, root_id in roots.items():
            if root_id in ancs:
                _terms_to_roots[term_id] = root_name
                break
        else:
            _terms_to_roots[term_id] = "other"
    return _terms_to_roots[term_id]

In [32]:
from linkml_runtime.dumpers import json_dumper


def coassoc_to_df(coassocs):
    objs = []
    for coassoc in coassocs:
        obj = json_dumper.to_dict(coassoc)
        obj["label1"] = go.label(obj["object1"])
        obj["label2"] = go.label(obj["object2"])
        obj["aspect1"] = get_term_category(obj["object1"])
        obj["aspect2"] = get_term_category(obj["object2"])
        objs.append(obj)
    return pd.DataFrame(objs)

df = coassoc_to_df(coassocs)
df

object1     object2  number_subjects_in_common  \
0    GO:0000003  GO:0000003                       1733   
1    GO:0002376  GO:0000003                         78   
2    GO:0002376  GO:0002376                        300   
3    GO:0003677  GO:0000003                        166   
4    GO:0003677  GO:0002376                         35   
..          ...         ...                        ...   
985  GO:0140110  GO:0051234                         11   
986  GO:0140110  GO:0071840                        193   
987  GO:0140110  GO:0071944                         14   
988  GO:0140110  GO:0098772                          5   
989  GO:0140110  GO:0140110                        736   

     proportion_subjects_in_common  number_subjects_in_union  \
0                         1.000000                      1733   
1                         0.039898                      1955   
2                         1.000000                       300   
3                         0.062571                      2653   
4                         0.025907                      1351   
..                             ...                       ...   
985                       0.004086                      2692   
986                       0.052008                      3711   
987                       0.005571                      2513   
988                       0.003544                      1411   
989                       1.000000                       736   

     number_subject_unique_to_entity1  number_subject_unique_to_entity2  \
0                                   0                                 0   
1                                 222                              1655   
2                                   0                                 0   
3                                 920                              1567   
4                                1051                               265   
..                                ...                               ...   
985                               725                              1956   
986                               543                              2975   
987                               722                              1777   
988                               731                               675   
989                                 0                                 0   

                               label1  \
0                        reproduction   
1               immune system process   
2               immune system process   
3                         DNA binding   
4                         DNA binding   
..                                ...   
985  transcription regulator activity   
986  transcription regulator activity   
987  transcription regulator activity   
988  transcription regulator activity   
989  transcription regulator activity   

                                            label2             aspect1  \
0                                     reproduction  biological_process   
1                                     reproduction  biological_process   
2                            immune system process  biological_process   
3                                     reproduction  molecular_function   
4                            immune system process  molecular_function   
..                                             ...                 ...   
985                  establishment of localization  molecular_function   
986  cellular component organization or biogenesis  molecular_function   
987                                 cell periphery  molecular_function   
988          molecular function regulator activity  molecular_function   
989               transcription regulator activity  molecular_function   

                aspect2  
0    biological_process  
1    biological_process  
2    biological_process  
3    biological_process  
4    biological_process  
..                  ...  
985  biological_process  
986  biological_process  
987  cellular_component  

In [42]:
filtered = df.query("object1 != object2")
filtered.sort_values(by="proportion_subjects_in_common", ascending=False)

object1     object2  number_subjects_in_common  \
656  GO:0045202  GO:0030054                        336   
697  GO:0046872  GO:0036094                       1063   
948  GO:0140110  GO:0003677                        567   
787  GO:0051234  GO:0005215                        868   
884  GO:0071944  GO:0016020                       1424   
..          ...         ...                        ...   
80   GO:0005773  GO:0003677                          0   
196  GO:0008283  GO:0005198                          0   
397  GO:0030246  GO:0008283                          0   
953  GO:0140110  GO:0005215                          0   
393  GO:0030246  GO:0006259                          0   

     proportion_subjects_in_common  number_subjects_in_union  \
656                       0.697095                       482   
697                       0.494649                      2149   
948                       0.451793                      1255   
787                       0.441281                      1967   
884                       0.424821                      3352   
..                             ...                       ...   
80                        0.000000                      1275   
196                       0.000000                       835   
397                       0.000000                       250   
953                       0.000000                      1604   
393                       0.000000                       461   

     number_subject_unique_to_entity1  number_subject_unique_to_entity2  \
656                                 0                               146   
697                                 0                              1086   
948                               169                               519   
787                              1099                                 0   
884                               367                              1561   
..                                ...                               ...   
80                                189                              1086   
196                               125                               710   
397                               125                               125   
953                               736                               868   
393                               125                               336   

                               label1                         label2  \
656                           synapse                  cell junction   
697                 metal ion binding         small molecule binding   
948  transcription regulator activity                    DNA binding   
787     establishment of localization           transporter activity   
884                    cell periphery                       membrane   
..                                ...                            ...   
80                            vacuole                    DNA binding   
196     cell population proliferation   structural molecule activity   
397              carbohydrate binding  cell population proliferation   
953  transcription regulator activity           transporter activity   
393              carbohydrate binding          DNA metabolic process   

                aspect1             aspect2  
656  cellular_component  cellular_component  
697  molecular_function  molecular_function  
948  molecular_function  molecular_function  
787  biological_process  molecular_function  
884  cellular_component  cellular_component  
..                  ...                 ...  
80   cellular_component  molecular_function  
196  biological_process  molecular_function  
397  molecular_function  biological_process  
953  molecular_function  molecular_function  
393  molecular_function  biological_process  

[946 rows x 11 columns]

In [43]:
def by_aspect(df, aspect):
    return df.query(f"aspect1 == '{aspect}' and aspect2 == '{aspect}'")

In [49]:
by_aspect(filtered, "biological_process").sort_values(by="proportion_subjects_in_common", ascending=False)

object1     object2  number_subjects_in_common  \
766  GO:0050896  GO:0023052                       1111   
406  GO:0032502  GO:0000003                       1174   
810  GO:0051234  GO:0033036                        686   
848  GO:0071840  GO:0032502                       1481   
321  GO:0019538  GO:0010467                       1024   
..          ...         ...                        ...   
611  GO:0044281  GO:0007049                          8   
718  GO:0050877  GO:0006259                          5   
795  GO:0051234  GO:0006259                         10   
614  GO:0044281  GO:0008283                          1   
610  GO:0044281  GO:0006259                          1   

     proportion_subjects_in_common  number_subjects_in_union  \
766                       0.401228                      2769   
406                       0.317555                      3697   
810                       0.313815                      2186   
848                       0.306943                      4825   
321                       0.303948                      3369   
..                             ...                       ...   
611                       0.005316                      1505   
718                       0.005313                       941   
795                       0.004361                      2293   
614                       0.001066                       938   
610                       0.000870                      1149   

     number_subject_unique_to_entity1  number_subject_unique_to_entity2  \
766                              1527                               131   
406                              1964                               559   
810                              1281                               219   
848                              1687                              1657   
321                              1495                               850   
..                                ...                               ...   
611                               806                               691   
718                               605                               331   
795                              1957                               326   
614                               813                               124   
610                               813                               335   

                                            label1  \
766                           response to stimulus   
406                          developmental process   
810                  establishment of localization   
848  cellular component organization or biogenesis   
321                      protein metabolic process   
..                                             ...   
611               small molecule metabolic process   
718                         nervous system process   
795                  establishment of localization   
614               small molecule metabolic process   
610               small molecule metabolic process   

                            label2             aspect1             aspect2  
766                      signaling  biological_process  biological_process  
406                   reproduction  biological_process  biological_process  
810     macromolecule localization  biological_process  biological_process  
848          developmental process  biological_process  biological_process  
321                gene expression  biological_process  biological_process  
..                             ...                 ...                 ...  
611                     cell cycle  biological_process  biological_process  
718          DNA metabolic process  biological_process  biological_process  
795          DNA metabolic process  biological_process  biological_process  
614  cell population proliferation  biological_process  biological_process  
610          DNA metabolic process  biological_process  biological_process  

[120 rows x 11 columns]

In [44]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))  # Adjust the size as needed


<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

In [45]:
import seaborn as sns

def plot_matrix(filtered: pd.DataFrame, aspect: str):
    pivoted = by_aspect(filtered, aspect).pivot(index="label1", columns="label2", values="proportion_subjects_in_common")
    ax = sns.heatmap(pivoted, annot=True, fmt=".2f")
    # ax.get_figure().savefig(f"output/{aspect}-heatmap.png", bbox_inches="tight"

plot_matrix(filtered, "biological_process")

/Users/cjm/Library/Caches/pypoetry/virtualenvs/oaklib-OeQZizwE-py3.9/lib/python3.9/site-packages/seaborn/matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)


In [52]:
from collections import defaultdict

coassoc_map = defaultdict(list)
def all_coassocs():
    for ribbon, taxa in ribbons.items():
        for taxon in taxa:
            pair = (ribbon, taxon)
            if pair not in coassoc_map:
                coassocs = fetch_matrix(taxon, ribbon)
                coassoc_map[pair] = coassocs

In [53]:
all_coassocs()

defaultdict(list,
            {('goslim_agr',
              'NCBITaxon:9606'): [PairwiseCoAssociation(object1='GO:0000003', object2='GO:0000003', object1_label=None, object2_label=None, number_subjects_in_common=1472, proportion_subjects_in_common=1.0, number_subjects_in_union=1472, number_subject_unique_to_entity1=0, number_subject_unique_to_entity2=0, subjects_in_common=[], associations_for_subjects_in_common=[]), PairwiseCoAssociation(object1='GO:0002376', object2='GO:0000003', object1_label=None, object2_label=None, number_subjects_in_common=201, proportion_subjects_in_common=0.055755894590846046, number_subjects_in_union=3605, number_subject_unique_to_entity1=2133, number_subject_unique_to_entity2=1271, subjects_in_common=[], associations_for_subjects_in_common=[]), PairwiseCoAssociation(object1='GO:0002376', object2='GO:0002376', object1_label=None, object2_label=None, number_subjects_in_common=2334, proportion_subjects_in_common=1.0, number_subjects_in_union=2334, number_subject

In [59]:
df = pd.DataFrame()
for pair, coassocs in coassoc_map.items():
    coassoc_df = coassoc_to_df(coassocs)
    filtered = coassoc_df.query("object1 != object2")
    df["ribbon"] = pair[0]
    df["taxon"] = pair[1]
    df = pd.concat([df, filtered])

In [60]:
df

ribbon           taxon     object1     object2  \
1    goslim_flybase_ribbon  NCBITaxon:7227  GO:0002376  GO:0000003   
3    goslim_flybase_ribbon  NCBITaxon:7227  GO:0003677  GO:0000003   
4    goslim_flybase_ribbon  NCBITaxon:7227  GO:0003677  GO:0002376   
6    goslim_flybase_ribbon  NCBITaxon:7227  GO:0003700  GO:0000003   
7    goslim_flybase_ribbon  NCBITaxon:7227  GO:0003700  GO:0002376   
..                     ...             ...         ...         ...   
984                    NaN             NaN  GO:0140110  GO:0050896   
985                    NaN             NaN  GO:0140110  GO:0051234   
986                    NaN             NaN  GO:0140110  GO:0071840   
987                    NaN             NaN  GO:0140110  GO:0071944   
988                    NaN             NaN  GO:0140110  GO:0098772   

     number_subjects_in_common  proportion_subjects_in_common  \
1                        201.0                       0.055756   
3                        311.0                       0.081649   
4                        276.0                       0.058649   
6                        144.0                       0.051010   
7                        124.0                       0.033468   
..                         ...                            ...   
984                      155.0                       0.048152   
985                       11.0                       0.004086   
986                      193.0                       0.052008   
987                       14.0                       0.005571   
988                        5.0                       0.003544   

     number_subjects_in_union  number_subject_unique_to_entity1  \
1                      3605.0                            2133.0   
3                      3809.0                            2337.0   
4                      4706.0                            2372.0   
6                      2823.0                            1351.0   
7                      3705.0                            1371.0   
..                        ...                               ...   
984                    3219.0                             581.0   
985                    2692.0                             725.0   
986                    3711.0                             543.0   
987                    2513.0                             722.0   
988                    1411.0                             731.0   

     number_subject_unique_to_entity2  \
1                              1271.0   
3                              1161.0   
4                              2058.0   
6                              1328.0   
7                              2210.0   
..                                ...   
984                            2483.0   
985                            1956.0   
986                            2975.0   
987                            1777.0   
988                             675.0   

                                        label1  \
1                        immune system process   
3                                  DNA binding   
4                                  DNA binding   
6    DNA-binding transcription factor activity   
7    DNA-binding transcription factor activity   
..                                         ...   
984           transcription regulator activity   
985           transcription regulator activity   
986           transcription regulator activity   
987           transcription regulator activity   
988           transcription regulator activity   

                                            label2             aspect1  \
1                                     reproduction  biological_process   
3                                     reproduction  molecular_function   
4                            immune system process  molecular_function   
6                                     reproduction  molecular_function   
7                            immune system process  molecular_function   
..                                             ...     

In [62]:
by_aspect(df, "molecular_function").sort_values(by="proportion_subjects_in_common", ascending=False)

ribbon           taxon     object1     object2  \
1170  goslim_flybase_ribbon  NCBITaxon:7227  GO:0046872  GO:0036094   
1170  goslim_flybase_ribbon  NCBITaxon:7227  GO:0046872  GO:0036094   
8     goslim_flybase_ribbon  NCBITaxon:7227  GO:0003700  GO:0003677   
8     goslim_flybase_ribbon  NCBITaxon:7227  GO:0003700  GO:0003677   
1170  goslim_flybase_ribbon  NCBITaxon:7227  GO:0046872  GO:0036094   
...                     ...             ...         ...         ...   
974   goslim_flybase_ribbon  NCBITaxon:7227  GO:0038023  GO:0008289   
972   goslim_flybase_ribbon  NCBITaxon:7227  GO:0038023  GO:0008134   
971   goslim_flybase_ribbon  NCBITaxon:7227  GO:0038023  GO:0008092   
954   goslim_flybase_ribbon  NCBITaxon:7227  GO:0038023  GO:0005215   
412   goslim_flybase_ribbon  NCBITaxon:7227  GO:0008289  GO:0005102   

      number_subjects_in_common  proportion_subjects_in_common  \
1170                     4347.0                       0.679113   
1170                     3687.0                       0.649463   
8                        1484.0                       0.558105   
8                        1323.0                       0.551250   
1170                      968.0                       0.540179   
...                         ...                            ...   
974                         0.0                       0.000000   
972                         0.0                       0.000000   
971                         0.0                       0.000000   
954                         0.0                       0.000000   
412                         0.0                       0.000000   

      number_subjects_in_union  number_subject_unique_to_entity1  \
1170                    6401.0                               0.0   
1170                    5677.0                               0.0   
8                       2659.0                              11.0   
8                       2400.0                              22.0   
1170                    1792.0                               0.0   
...                        ...                               ...   
974                      169.0                              16.0   
972                       92.0                              16.0   
971                      137.0                              16.0   
954                      474.0                              16.0   
412                      161.0                             153.0   

      number_subject_unique_to_entity2  \
1170                            2054.0   
1170                            1990.0   
8                               1164.0   
8                               1055.0   
1170                             824.0   
...                                ...   
974                              153.0   
972                               76.0   
971                              121.0   
954                              458.0   
412                                8.0   

                                         label1                        label2  \
1170                          metal ion binding        small molecule binding   
1170                          metal ion binding        small molecule binding   
8     DNA-binding transcription factor activity                   DNA binding   
8     DNA-binding transcription factor activity                   DNA binding   
1170                          metal ion binding        small molecule binding   
...                                         ...                           ...   
974                 signaling receptor activity                 lipid binding   
972                 signaling receptor activity  transcription factor binding   
971                 signaling receptor activity  cytoskeletal protein binding   
954                 signaling receptor activity          transporter activity   
412                               lipid binding    signaling receptor binding   

                 aspect1             aspect2  
1170  molecular_function  molecul

In [63]:
by_aspect(df, "biological_process").sort_values(by="proportion_subjects_in_common", ascending=False)


ribbon           taxon     object1     object2  \
856   goslim_flybase_ribbon  NCBITaxon:7227  GO:0032502  GO:0030154   
856   goslim_flybase_ribbon  NCBITaxon:7227  GO:0032502  GO:0030154   
1259  goslim_flybase_ribbon  NCBITaxon:7227  GO:0050896  GO:0023052   
1259  goslim_flybase_ribbon  NCBITaxon:7227  GO:0050896  GO:0023052   
856   goslim_flybase_ribbon  NCBITaxon:7227  GO:0032502  GO:0030154   
...                     ...             ...         ...         ...   
379   goslim_flybase_ribbon  NCBITaxon:7227  GO:0008283  GO:0002376   
402   goslim_flybase_ribbon  NCBITaxon:7227  GO:0008283  GO:0007610   
1177  goslim_flybase_ribbon  NCBITaxon:7227  GO:0050877  GO:0002376   
1200  goslim_flybase_ribbon  NCBITaxon:7227  GO:0050877  GO:0007610   
1203  goslim_flybase_ribbon  NCBITaxon:7227  GO:0050877  GO:0008283   

      number_subjects_in_common  proportion_subjects_in_common  \
856                      4001.0                       0.647935   
856                      3756.0                       0.635641   
1259                     5698.0                       0.579183   
1259                     4987.0                       0.576399   
856                       116.0                       0.552381   
...                         ...                            ...   
379                         0.0                            NaN   
402                         0.0                            NaN   
1177                        0.0                            NaN   
1200                        0.0                            NaN   
1203                        0.0                            NaN   

      number_subjects_in_union  number_subject_unique_to_entity1  \
856                     6175.0                            2174.0   
856                     5909.0                            2153.0   
1259                    9838.0                            3977.0   
1259                    8652.0                            3473.0   
856                      210.0                              94.0   
...                        ...                               ...   
379                        0.0                               0.0   
402                        0.0                               0.0   
1177                       0.0                               0.0   
1200                       0.0                               0.0   
1203                       0.0                               0.0   

      number_subject_unique_to_entity2                         label1  \
856                                0.0          developmental process   
856                                0.0          developmental process   
1259                             163.0           response to stimulus   
1259                             192.0           response to stimulus   
856                                0.0          developmental process   
...                                ...                            ...   
379                                0.0  cell population proliferation   
402                                0.0  cell population proliferation   
1177                               0.0         nervous system process   
1200                               0.0         nervous system process   
1203                               0.0         nervous system process   

                             label2             aspect1             aspect2  
856            cell differentiation  biological_process  biological_process  
856            cell differentiation  biological_process  biological_process  
1259                      signaling  biological_process  biological_process  
1259                      signaling  biological_process  biological_process  
856            cell differentiation  biological_process  biological_process  
...                             ...                 ...                 ...  
379           immune system process  biological_process  biological_process  
402                        behavior  biological_process  

In [64]:
by_aspect(df, "cellular_component").sort_values(by="proportion_subjects_in_common", ascending=False)

ribbon           taxon     object1     object2  \
1116  goslim_flybase_ribbon  NCBITaxon:7227  GO:0045202  GO:0030054   
656                     NaN             NaN  GO:0045202  GO:0030054   
1116  goslim_flybase_ribbon  NCBITaxon:7227  GO:0045202  GO:0030054   
793   goslim_flybase_ribbon  NCBITaxon:7227  GO:0031410  GO:0005768   
793   goslim_flybase_ribbon  NCBITaxon:7227  GO:0031410  GO:0005768   
...                     ...             ...         ...         ...   
1090  goslim_flybase_ribbon  NCBITaxon:7227  GO:0045202  GO:0005576   
1070  goslim_flybase_ribbon  NCBITaxon:7227  GO:0042995  GO:0030054   
100   goslim_flybase_ribbon  NCBITaxon:7227  GO:0005768  GO:0005576   
1098  goslim_flybase_ribbon  NCBITaxon:7227  GO:0045202  GO:0005829   
1116  goslim_flybase_ribbon  NCBITaxon:7227  GO:0045202  GO:0030054   

      number_subjects_in_common  proportion_subjects_in_common  \
1116                     1644.0                       0.717904   
656                       336.0                       0.697095   
1116                     1484.0                       0.662796   
793                       200.0                       0.623053   
793                      1030.0                       0.473346   
...                         ...                            ...   
1090                        0.0                       0.000000   
1070                        0.0                       0.000000   
100                         0.0                       0.000000   
1098                        0.0                       0.000000   
1116                        0.0                            NaN   

      number_subjects_in_union  number_subject_unique_to_entity1  \
1116                    2290.0                               0.0   
656                      482.0                               0.0   
1116                    2239.0                               0.0   
793                      321.0                             121.0   
793                     2176.0                            1146.0   
...                        ...                               ...   
1090                     131.0                               0.0   
1070                     136.0                             136.0   
100                      331.0                             200.0   
1098                    1298.0                               0.0   
1116                       0.0                               0.0   

      number_subject_unique_to_entity2               label1  \
1116                             646.0              synapse   
656                              146.0              synapse   
1116                             755.0              synapse   
793                                0.0  cytoplasmic vesicle   
793                                0.0  cytoplasmic vesicle   
...                                ...                  ...   
1090                             131.0              synapse   
1070                               0.0      cell projection   
100                              131.0             endosome   
1098                            1298.0              synapse   
1116                               0.0              synapse   

                    label2             aspect1             aspect2  
1116         cell junction  cellular_component  cellular_component  
656          cell junction  cellular_component  cellular_component  
1116         cell junction  cellular_component  cellular_component  
793               endosome  cellular_component  cellular_component  
793               endosome  cellular_component  cellular_component  
...                    ...                 ...                 ...  
1090  extracellular region  cellular_component  cellular_component  
1070         cell junction  cellular_component  cellular_component  
100   extracellular region  cellular_component  cellular_component  
1098               cytosol  cellular_component  cellular_component  
1116         cell junction  cellular_compo